This network uses a hybrid architecture combining several modern techniques:

1. Inception Modules - Multi-scale feature extraction through parallel 
   convolutional paths of different kernel sizes. This allows the network
   to capture patterns at multiple scales simultaneously.

2. Residual Connections - Skip connections that help gradients flow through
   deep networks and allow the network to learn residual mappings.

3. Squeeze-and-Excitation Blocks - Channel attention mechanism that learns
   to weight feature channels based on their importance for the task.

4. Dense Connections - Cross-stage connections that allow features from
   earlier layers to directly influence later layers, improving gradient
   flow and feature reuse.

5. Dual Attention - Combined spatial and channel attention in the final
   stages to focus on important regions and features.

6. Multi-Scale Pooling - Uses both average and max pooling to capture
   different statistical properties of the feature maps.

The architecture processes features through three main stages with 
progressively increasing complexity and abstraction. Early stages use
inception modules for multi-scale processing, middle stages use residual
learning, and late stages apply attention mechanisms before classification.

In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In this section you can experiment with whatever ConvNet architecture you'd like on CIFAR-10.

You should experiment with architectures, hyperparameters, loss functions, or anything else you can think of to train a model that achieves **at least 70%** accuracy on the **Test** set within 30 epochs.

In [ ]:
# Load CIFAR-10 dataset
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.cifar10.load_data()

In [ ]:

# Normalize pixel values to [0, 1]
x_train = x_train0.reshape(50000, 32, 32, 3).astype("float32") / 255.0
x_test = x_test0.reshape(10000, 32, 32, 3).astype("float32") / 255.0
y_train = np.asarray(y_train0, dtype=np.int32)
y_test = np.asarray(y_test0, dtype=np.int32)

In [ ]:
def inception_block(x, filters):
    # Inception module with 4 parallel paths for multi-scale feature extraction
    
    # Path 1: 1x1 convolution for dimensionality reduction
    path1 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    
    # Path 2: 1x1 reduction followed by 3x3 convolution
    path2 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    path2 = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(path2)
    
    # Path 3: 1x1 reduction followed by two 3x3 convolutions (simulates 5x5)
    path3 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    path3 = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(path3)
    path3 = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(path3)
    
    # Path 4: Max pooling followed by 1x1 convolution
    path4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    path4 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(path4)
    
    # Concatenate all paths along the channel dimension
    return layers.Concatenate()([path1, path2, path3, path4])

In [ ]:
def residual_block(x, filters):
    # Residual block with skip connection for gradient flow
    shortcut = x
    
    # Main path: two conv layers with batch normalization
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Adjust shortcut dimensions if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    # Add shortcut to main path and apply activation
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
def squeeze_excitation_block(x, ratio=16):
    # Channel attention mechanism using squeeze-and-excitation
    channels = x.shape[-1]
    
    # Squeeze: global spatial information into channel descriptor
    se = layers.GlobalAveragePooling2D()(x)
    
    # Excitation: learn channel-wise weights
    se = layers.Dense(channels // ratio, activation='relu')(se)
    se = layers.Dense(channels, activation='sigmoid')(se)
    
    # Scale: apply learned weights to input
    se = layers.Reshape((1, 1, channels))(se)
    return layers.Multiply()([x, se])

In [ ]:
def MyNet():
    # Build hybrid architecture combining inception, residual, and attention mechanisms
    
    inputs = layers.Input(shape=(32, 32, 3))
    
    # Data augmentation applied during training only
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.15)(x)
    x = layers.RandomZoom(0.1)(x)
    x = layers.RandomContrast(0.2)(x)
    
    # Initial feature extraction
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 1: Multi-scale feature extraction at 32x32 resolution
    inception1 = inception_block(x, 16)
    inception1 = layers.BatchNormalization()(inception1)
    inception1 = layers.Dropout(0.2)(inception1)
    
    residual1 = residual_block(inception1, 64)
    se1 = squeeze_excitation_block(residual1)
    
    # Reduce spatial dimensions to 16x16
    x = layers.MaxPooling2D((2, 2))(se1)
    
    # Stage 2: Deeper feature learning at 16x16 resolution
    inception2 = inception_block(x, 24)
    inception2 = layers.BatchNormalization()(inception2)
    inception2 = layers.Dropout(0.3)(inception2)
    
    residual2 = residual_block(inception2, 96)
    se2 = squeeze_excitation_block(residual2)
    
    # Dense connection: add features from stage 1 to stage 2
    se1_pooled = layers.MaxPooling2D((2, 2))(se1)
    se1_adjusted = layers.Conv2D(96, (1, 1), padding='same')(se1_pooled)
    dense_concat1 = layers.Add()([se2, se1_adjusted])
    
    # Reduce spatial dimensions to 8x8
    x = layers.MaxPooling2D((2, 2))(dense_concat1)
    
    # Stage 3: Deep feature processing at 8x8 resolution
    residual3a = residual_block(x, 128)
    residual3a = layers.Dropout(0.35)(residual3a)
    
    residual3b = residual_block(residual3a, 128)
    se3 = squeeze_excitation_block(residual3b)
    
    # Dual attention mechanism for comprehensive feature refinement
    # Path A: spatial attention to focus on important regions
    spatial_attention = layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')(se3)
    spatial_features = layers.Multiply()([se3, spatial_attention])
    
    # Path B: additional channel-wise transformation
    channel_features = layers.Conv2D(128, (1, 1), activation='relu')(se3)
    
    # Combine both attention paths
    x = layers.Concatenate()([spatial_features, channel_features])
    x = layers.Conv2D(256, (1, 1), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    # Final feature compression
    x = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Multi-scale global pooling to capture both average and max activations
    gap = layers.GlobalAveragePooling2D()(x)
    gmp = layers.GlobalMaxPooling2D()(x)
    x = layers.Concatenate()([gap, gmp])
    
    # Classification head
    x = layers.Dense(384, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(192, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model with Adam optimizer
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [ ]:
# Create and display model
model = MyNet()
model.summary()
print(f"\nTotal parameters: {model.count_params():,}")

# Setup training callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1)
]

# Train the model
print("\nStarting training...")
training_history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Plot training curves
print("\n" + "="*70)
print("TRAINING HISTORY")
print("="*70)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(training_history.history['accuracy']) + 1)

# Accuracy plot
ax1.plot(epochs_range, training_history.history['accuracy'], 
         label='Training', linewidth=2, marker='o', markersize=3)
ax1.plot(epochs_range, training_history.history['val_accuracy'], 
         label='Validation', linewidth=2, marker='s', markersize=3)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(epochs_range, training_history.history['loss'], 
         label='Training', linewidth=2, marker='o', markersize=3)
ax2.plot(epochs_range, training_history.history['val_loss'], 
         label='Validation', linewidth=2, marker='s', markersize=3)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()


# Evaluate on test set
print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

if test_accuracy >= 0.70:
    print(f"Target achieved (>= 70%)")
else:
    print(f"Target not met - need {(0.70 - test_accuracy)*100:.2f}% more")


# Save model architecture diagram
print("\n" + "="*70)
print("SAVING NETWORK DIAGRAM")
print("="*70)

try:
    plot_model(model, to_file='model_architecture.png', 
               show_shapes=True, show_layer_names=True, 
               rankdir='TB', dpi=150)
    print("Network diagram saved to 'model_architecture.png'")
except Exception as e:
    print(f"Could not save diagram: {e}")
    print("Install graphviz and pydot if needed")


# Sample predictions
print("\n" + "="*70)
print("SAMPLE PREDICTIONS")
print("="*70)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    idx = np.random.randint(0, len(x_test))
    prediction = model.predict(x_test[idx:idx+1], verbose=0)
    pred_label = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    true_label = y_test[idx][0]
    
    axes[i].imshow(x_test0[idx])
    axes[i].axis('off')
    
    # Color code correct vs incorrect predictions
    color = 'green' if pred_label == true_label else 'red'
    axes[i].set_title(f'True: {class_names[true_label]}\nPred: {class_names[pred_label]} ({confidence:.1f}%)',
                     color=color, fontsize=9)

plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=150)
plt.show()

print("\nTraining complete.")
print(f"Final test accuracy: {test_accuracy*100:.2f}%")
print(f"Total parameters: {model.count_params():,}")

In [ ]:
# for i, img in enumerate(x_test0[0:64]):
#     plt.subplot(8, 8, i+1)
#     plt.imshow(img, cmap='gray')
#     plt.axis('off')
#     #prediction = model.predict(x_test[i:i+1])
#     #pred_label = np.argmax(prediction)
    
   
#     plt.tight_layout()

In [ ]:
# def MyNet():
#     # your code here
#     model = tf.keras.Sequential([
       
#     ])
#     # you can modify the optimizer as needed
#     model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
#     return model

  

In [ ]:
# model = MyNet()
# training_history = model.fit(x_train, y_train, epochs=10)

In the cell below:  

1) Plot the network diagram
2) Write an explanation of what you did
3) Plot the loss and accuracy graph over epoches
4) Report accuracy on Test dataset. 